In [1]:
import numpy as np
from scipy.optimize import minimize

class SupplyChainPH:
    def __init__(self, num_facilities, num_customers, num_scenarios, fixed_costs, transport_costs, capacities, demands, probabilities, rho=1.0):
        self.num_facilities = num_facilities
        self.num_customers = num_customers
        self.num_scenarios = num_scenarios
        self.fixed_costs = fixed_costs
        self.transport_costs = transport_costs
        self.capacities = capacities
        self.demands = demands
        self.probabilities = probabilities
        self.rho = rho

        # Initialize variables
        self.x = np.random.rand(num_scenarios, num_facilities, num_customers)
        self.w = np.zeros((num_scenarios, num_facilities, num_customers))
        self.z = np.mean(self.x, axis=0)

    def scenario_subproblem(self, scenario):
        def objective(x):
            x_reshaped = x.reshape(self.num_facilities, self.num_customers)
            fixed_cost = np.sum(self.fixed_costs * (np.sum(x_reshaped, axis=1) > 0))
            transport_cost = np.sum(self.transport_costs * x_reshaped)
            augmented_lagrangian = np.sum(self.w[scenario] * (x_reshaped - self.z)) + \
                                   (self.rho/2) * np.sum((x_reshaped - self.z)**2)
            return fixed_cost + transport_cost + augmented_lagrangian

        def constraints(x):
            x_reshaped = x.reshape(self.num_facilities, self.num_customers)
            capacity_constraint = self.capacities - np.sum(x_reshaped, axis=1)
            demand_constraint = np.sum(x_reshaped, axis=0) - self.demands[scenario]
            return np.concatenate([capacity_constraint, demand_constraint])

        x0 = self.x[scenario].flatten()
        cons = {'type': 'ineq', 'fun': constraints}
        bounds = [(0, None) for _ in range(self.num_facilities * self.num_customers)]

        result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)
        return result.x.reshape(self.num_facilities, self.num_customers)

    def calculate_objective(self, x):
        fixed_cost = np.sum(self.fixed_costs * (np.sum(x, axis=1) > 0))
        transport_cost = np.sum(self.transport_costs * x)
        return fixed_cost + transport_cost

    def solve(self, max_iterations=100, tolerance=1e-4):
        for iteration in range(max_iterations):
            # Solve scenario subproblems
            for s in range(self.num_scenarios):
                self.x[s] = self.scenario_subproblem(s)

            # Compute new average solution
            z_new = np.average(self.x, axis=0, weights=self.probabilities)

            # Calculate and print objective value
            obj_value = np.sum([self.probabilities[s] * self.calculate_objective(self.x[s]) for s in range(self.num_scenarios)])
            print(f"Iteration {iteration+1}, Objective Value: {obj_value:.2f}")

            # Update price variables
            for s in range(self.num_scenarios):
                self.w[s] += self.rho * (self.x[s] - z_new)

            # Check convergence
            if np.max(np.abs(z_new - self.z)) < tolerance:
                break

            self.z = z_new

        final_obj_value = self.calculate_objective(self.z)
        print(f"\nFinal Objective Value: {final_obj_value:.2f}")
        return self.z

# Problem parameters
num_facilities = 3
num_customers = 5
num_scenarios = 100

# Fixed costs for opening facilities
fixed_costs = np.array([100000, 120000, 90000])

# Transportation costs from facilities to customers
trans_costs = np.array([
    [10, 15, 20, 25, 30],
    [12, 14, 18, 22, 26],
    [8, 16, 24, 32, 40]
])

# Facility capacities
capacities = np.array([5000, 6000, 4500])

# Generate random demand scenarios
np.random.seed(42)
demand_mean = 1000
demand_std = 200
demand_scenarios = np.random.normal(demand_mean, demand_std, (num_scenarios, num_customers))
demand_scenarios = np.clip(demand_scenarios, 0, None)  # Ensure non-negative demand

# Equal probabilities for all scenarios
probabilities = np.ones(num_scenarios) / num_scenarios

# Create and solve the problem
ph_solver = SupplyChainPH(num_facilities, num_customers, num_scenarios, 
                          fixed_costs, trans_costs, capacities, demand_scenarios, probabilities)

solution = ph_solver.solve(max_iterations=200, tolerance=1e-3)

print("\nOptimal facility allocations:")
print(solution)
print("\nFacilities to open:")
print(np.where(np.sum(solution, axis=1) > 0)[0])

# Calculate average allocation per facility
avg_allocation = np.sum(solution, axis=1)
print("\nAverage allocation per facility:")
for i, alloc in enumerate(avg_allocation):
    print(f"Facility {i}: {alloc:.2f}")

# Calculate average demand satisfaction
avg_demand = np.mean(demand_scenarios, axis=0)
avg_supply = np.sum(solution, axis=0)
print("\nAverage demand vs supply per customer:")
for i, (demand, supply) in enumerate(zip(avg_demand, avg_supply)):
    print(f"Customer {i}: Demand = {demand:.2f}, Supply = {supply:.2f}")

Iteration 1, Objective Value: 414036.04
Iteration 2, Objective Value: 426747.17
Iteration 3, Objective Value: 435667.35
Iteration 4, Objective Value: 442221.88
Iteration 5, Objective Value: 446942.74
Iteration 6, Objective Value: 450504.59
Iteration 7, Objective Value: 453246.23
Iteration 8, Objective Value: 455308.74
Iteration 9, Objective Value: 456818.14
Iteration 10, Objective Value: 457856.48
Iteration 11, Objective Value: 458500.93
Iteration 12, Objective Value: 458878.89
Iteration 13, Objective Value: 459070.76
Iteration 14, Objective Value: 459116.22
Iteration 15, Objective Value: 459049.94
Iteration 16, Objective Value: 458891.56
Iteration 17, Objective Value: 458688.11
Iteration 18, Objective Value: 458451.21
Iteration 19, Objective Value: 458180.52
Iteration 20, Objective Value: 457883.31
Iteration 21, Objective Value: 457676.72
Iteration 22, Objective Value: 457579.22
Iteration 23, Objective Value: 457581.14
Iteration 24, Objective Value: 457666.82
Iteration 25, Objective V

In [4]:
solution

array([[4.89096437e+02, 4.56067006e+02, 5.00514588e+02, 4.83762035e+02,
        4.95175617e+02],
       [8.90662927e+01, 6.60608877e+02, 8.99412600e+02, 1.08223079e+03,
        1.29512595e+03],
       [8.89106473e+02, 2.56619702e+02, 3.85418995e-08, 8.12962758e-11,
        1.28707516e-10]])